#  Kmeans Clustering on Text Files

In [392]:
import numpy as np
import pandas as pd
from sklearn import svm
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy import spatial
import os
from collections import Counter
np.random.seed(42)

The function below calculates distance between two data points.Tried for various distanca metrics like cosine distance,sine distance,eucledian distance,but only eucledian gave proper results.


In [393]:
def distance(p1,p2):
    #print(1 - spatial.distance.cosine(p1,p2))
    #return 1 - spatial.distance.cosine(p1,p2)
    return np.sqrt(np.sum((p1-p2)**2))

Finds the nearest centroid that point belongs to.

In [394]:
def closest_centroid(sample,centroids):
    distances=[]
    for point in centroids:
        d=distance(sample,point)
        distances.append(d)
    closest=np.argmin(distances)
    return closest

Creates clusters/assigns points to clusters.Populates clusters.

In [395]:
def create_clusters(X_train,K,centroids):
    clusters=[[] for _ in range(K)]
    for index,sample in enumerate(X_train):
        centroid_index=closest_centroid(sample,centroids)
        clusters[centroid_index].append(index)
    return clusters

Finds centroids of the current collection of data.

In [396]:
def get_centroids(clusters,K,num_features,X_train):
    centroids=np.zeros((K,num_features))
    for cluster_index,cluster in enumerate(clusters):
        cluster_mean=np.mean(X_train[cluster],axis=0)
        centroids[cluster_index]=cluster_mean
    return centroids    

In [427]:
def predict(K,X_train,num_iterations,plot_steps=False):
    clusters=[[] for _ in range(K)]
    centroids=[]
    num_samples=X_train.shape[0]
    num_features=X_train.shape[1]
    cnt=0
    #initializing centroids between 0 and number of samples
    random_indexes=np.random.choice(num_samples,K,replace=False)
    #random_indexes=[ 191 , 183  , 84  ,797 ,1695]
    #random_indexes=kmeanspp(X_train,K)
    #191  183   84  797 1695
    #[ 795  651 1091  782 1065]
    #1386,91,1061,1144,444
    centroids=[X_train[index] for index in random_indexes]
    print(random_indexes)
    while(cnt<=num_iterations):
        cnt+=1
        clusters=create_clusters(X_train,K,centroids)
        centroids_old=centroids
        centroids=get_centroids(clusters,K,num_features,X_train)
        #centroids=kmeanspp(X_train,K)
        #if(cnt%10==0):    
            #print(cnt)
        completed=[]
        for i in range(K):
            d=distance(centroids_old[i],centroids[i])
            completed.append(d)
        if(sum(completed)==0):
            break
    #return get_cluster_labels(clusters,num_samples)
    return clusters

In [401]:
def cleanpunc(sentence):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in sentence if ch not in exclude)

In [403]:
file_names=os.listdir(r'C:\Users\Rishab\Assignment-2_Dataset\Datasets\Question-6\dataset')
len(file_names)

1725

In [404]:
text_data={'files':list(), 'data':list()}

In [405]:
for filename in file_names:
    with open(r'C:\Users\Rishab\Assignment-2_Dataset\Datasets\Question-6\dataset\\'+filename,"r") as data:
        if filename in text_data:
            continue
        text_data['files'].append(filename)
        text_data['data'].append(data.read())

In [406]:
labels=[]
for name in file_names:
    label=name.split('_')
    label=label[1].split('.')
    labels.append(label[0])

In [407]:
text_data['labels']=labels

In [408]:
df=pd.DataFrame(text_data)
df.head()

,files,data,labels
0,100_1.txt,Shares rise on new Man Utd offer\n\nShares in ...,1
1,100_2.txt,Slater to star in Broadway play\n\nActor Chris...,2
2,100_3.txt,Galloway plea for hostage release\n\nEx-Labour...,3
3,100_4.txt,Connors boost for British tennis\n\nFormer wor...,4
4,100_5.txt,Moving mobile improves golf swing\n\nA mobile ...,5


The following functions will be used in preprocessing the text,the functions will be taking input as texts and will remove the punctuations,and change all of them to lower case for uniformness.The data henced process will be used for further stages of classification.

In [409]:
import re
import string
i=0
list_of_sentences=[]
for sent in df['data'].values:
    filtered_sentence=[]
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sentences.append(filtered_sentence)
    
    df.loc[i,'filtered_data']=' '.join(filtered_sentence)
    i+=1

In [410]:
df

,files,data,labels,filtered_data
0,100_1.txt,Shares rise on new Man Utd offer\n\nShares in ...,1,shares rise on new man utd offer shares in man...
1,100_2.txt,Slater to star in Broadway play\n\nActor Chris...,2,slater to star in broadway play actor christia...
2,100_3.txt,Galloway plea for hostage release\n\nEx-Labour...,3,galloway plea for hostage release exlabour mp ...
3,100_4.txt,Connors boost for British tennis\n\nFormer wor...,4,connors boost for british tennis former world ...
4,100_5.txt,Moving mobile improves golf swing\n\nA mobile ...,5,moving mobile improves golf swing a mobile pho...
...,...,...,...,...
1720,9_1.txt,Saudi investor picks up the Savoy\n\nLondon's ...,1,saudi investor picks up the savoy londons famo...
1721,9_2.txt,Surprise win for anti-Bush film\n\nMichael Moo...,2,surprise win for antibush film michael moores ...
1722,9_3.txt,Correction agency plans dropped\n\nPlans to cr...,3,correction agency plans dropped plans to creat...
1723,9_4.txt,Cole refuses to blame van Persie\n\nAshley Col...,4,cole refuses to blame van persie ashley cole h...


In [411]:
actual=df['labels'].values

In [412]:
df['filtered_data'].iloc[0]

'shares rise on new man utd offer shares in manchester united closed up on monday following a new offer from us tycoon malcolm glazer the board of the football club is expected to meet early this week to discuss the latest proposal which values the club at manchester united revealed on sunday that it had received a detailed proposal from mr glazer which looks set to receive more serious scrutiny the club has previously rejected mr glazers approaches out of hand but a senior source at the club told the bbc this time its different supporters group shareholders united however urged the club to reject the new deal a spokesman for the shareholders united said i cant see any difference compared to mr glazers previous proposals other than less debt he isnt bringing any money into the club hell use our money to buy it mr glazers latest move is being led by mr glazers two sons avi and joel according to the financial times a proposal was received by david gill uniteds chief executive at the end 

In [413]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_vect = TfidfVectorizer(stop_words='english')
Train_X_Tfidf=Tfidf_vect.fit_transform(df['filtered_data'])
Train_X_Tfidf=np.array(Train_X_Tfidf.todense())
#print(count_vectorizer.get_feature_names())

In [414]:
Train_X_Tfidf.shape

(1725, 28020)

The numpy array now has more dimensions than csr representation,but it is how my model was built.It only handles numpy array as input.

In [415]:
Train_X_Tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

It is a well known fact that Kmeans clustering is sensitive to initalization.
The initial clusters chosen tremendously effects the cluster formation/detection.
Hence while experimenting one of the settings i.e 191  183   84  797 1695 showed the best accuracy.

The best clustering accuracy was found when the initial centroids were chosen as [191  183   84  797 1695] indices.
It showed 91.2 % accuracy.

In [426]:
num_clusters =5
for _ in range(1):
    clusters=predict(num_clusters,Train_X_Tfidf,200,plot_steps=False)
    klst=[1,2,3,4,5]
    cluster_labels=[]
    for k in klst:
        ans = Counter(actual[clusters[k-1]]).most_common()
        cluster_labels.append(ans[0][0])
    final_labels=[0 for _ in range(len(actual))]
    for i in range(len(clusters)):
        for j in clusters[i]:
            final_labels[j] = cluster_labels[i]
    print(np.sum(final_labels==actual)/len(actual))


[191, 183, 84, 797, 1695]
0.912463768115942


The following run is when clusters are initialized randomly and hence we see varying accuracy results.

In [428]:
num_clusters =5
for _ in range(5):
    clusters=predict(num_clusters,Train_X_Tfidf,200,plot_steps=False)
    klst=[1,2,3,4,5]
    cluster_labels=[]
    for k in klst:
        ans = Counter(actual[clusters[k-1]]).most_common()
        cluster_labels.append(ans[0][0])
    final_labels=[0 for _ in range(len(actual))]
    for i in range(len(clusters)):
        for j in clusters[i]:
            final_labels[j] = cluster_labels[i]
    print(np.sum(final_labels==actual)/len(actual))

[ 126  717 1366  491  819]
0.6272463768115942
[ 900  408 1202  192  366]
0.823768115942029
[1608  369 1124 1150 1299]
0.758840579710145
[1558  552  315  594 1595]
0.7547826086956522
[ 308  165 1225 1137 1128]
0.5814492753623188
